# HPC Analytics Template Notebook

## Basic Idea
- Create a private repo
- submodule hpc_analytics into that repo
- Copy this template into that notebook
- Customize as need be...
- run reports


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy
import scipy
import math
import matplotlib as mpl
import matplotlib.dates as mpd
import pylab as plt
import datetime as dtm
import pytz
import multiprocessing as mpp
import pickle
import sys
import os
import h5py
#import socket
#import lmod
# lmod.load('system')
# lmod.load('texlive')
# lmod.
#
# TODO: phase out unreferenced hpc_lib calls...
import HPC_analytics.hpc_lib as hpc_lib
GB=1024.**3.
#import HPC_analytics.hpc_reports as hpc_reports
#
# def running_mean(X,n=10):
#     return (numpy.cumsum(numpy.insert(X,0,0))[n:] - numpy.cumsum(numpy.insert(X,0,0))[:-n])/n
# #

In [ ]:
# import socket
# socket.gethostname()
# CPU cores: 7712
# GPU cores: 1344?

In [ ]:
n_cpus = 4
#print('** epoch: {}'.format(mpd.get_epoch()))
if 'SLURM_CPUS_PER_TASK' in os.environ.keys():
    n_cpus = int(os.environ['SLURM_CPUS_PER_TASK'])
#
print(f'*** n_cpus: {n_cpus}')

In [ ]:
N_report_len = 180
end_dtm = dtm.datetime(2025,7,15)
end_date = end_dtm.date()
start_date = end_date - dtm.timedelta(days=N_report_len)
print('*** dates: {} - {}'.format(start_date, end_date))
#delim_sacct='|'
delim_sacct='*'
#partition='normal'
partition='hns'
#partition=None
group=None
s_user=None
verbose=0
# group='oneillm'
# group='edunham'
#s_user = 'labraha2'
#
SACCT_data_path = os.path.join(os.environ['SCRATCH'], 'SACCT', 'sacct_data')
#SACCT_data_path='./sacct_data'
sacct_h5_file_name = f'sacct_sherlock_p{str(partition).upper()}_g{str(group).upper()}_u{s_user}_{start_date.year:04}{start_date.month:02}{start_date.day:02}_{end_dtm.year:04}{end_dtm.month:02}{end_dtm.day:02}.h5'
#
sacct_h5_file = os.path.join(SACCT_data_path, sacct_h5_file_name)
print(f'*** sacct_h5_file [{os.path.isfile(sacct_h5_file)}]: {sacct_h5_file}')

In [ ]:
print('** ', sacct_h5_file)
print('** ', os.path.split(sacct_h5_file))


In [ ]:
# NOTE: to get individual users, eg to get Eric's group usage:
#. sacct --allusers --user=labraha2 --start=2022-01-01 --end=2022-06-15
#
# NOTE: additional options can be passed in the more_options=[] arrary, or just as sacct_{option-name}={val}
#
if os.path.isfile(sacct_h5_file):
    print('*** Creating SACCT object from HDF5 {}'.format(sacct_h5_file))
    SACCT_obj = hpc_lib.SACCT_data_from_h5(sacct_h5_file, keep_raw_data=False, n_cpu=n_cpus)
    #
    
else:
    print('*** Fetching SACCT data directly')
    SACCT_obj = hpc_lib.SACCT_data_direct(group=group, partition=partition, start_date=str(start_date),
                                          n_cpu=n_cpus, verbose=verbose, delim=delim_sacct,
                                          end_date=str(end_date), keep_raw_data=False)
    # , sacct_user=s_user
    print(f'** writing HDF5: {sacct_h5_file}')
    pth, fn = os.path.split(sacct_h5_file)
    if not os.path.isdir(pth):
        os.makedirs(pth)
    
    SACCT_obj.write_hdf5(sacct_h5_file)
    #
#
print('** ', SACCT_obj.jobs_summary.dtype.names)

In [ ]:
#import h5py
print('** ', h5py)

In [ ]:
SP=hpc_lib.SH_PART_obj()
#
# SH_PART will only show partitions of which caller/user is a member. For more general applications, use
#. SINFO_obj()
SIo = hpc_lib.SINFO_obj(partition=partition, Format_fields='node')

In [ ]:
print('** ', SIo.total_cpus)

In [ ]:
print('** cols: ', SIo.sinfo_data.dtype.names)

In [ ]:
total_cpus  = SIo.total_cpus
total_nodes = SIo.total_nodes
total_gpus  = SIo.get_total_cpus('GPUS')


print(f'** partition: {partition}')
print(f'*** Cols: {SACCT_obj.jobs_summary.dtype.names}')
print(f'*** CPUs: {total_cpus}')
print(f'*** Nodes: {total_nodes}')
#
# note, all methods of computing N_GPUs appear to agree
print(f'*** GPUs: {total_gpus}, {numpy.sum(SIo.sinfo_data["GPUS"])}, {SIo.get_total_cpus("GPUS")}')

unique_users = numpy.unique(SACCT_obj['User'])
print(f'** len: {len(unique_users)} users')

In [ ]:
#n_cpus_serc = SP.get_total_cpus(partitions=partition)
#n_gpus_serc = SP.get_total_gpus(partitions=partition)

n_cpus_serc = total_cpus
n_gpus_serc = total_gpus

#
print(f'** n_cpus: {n_cpus_serc}, n_gpus: {n_gpus_serc}')
#
#print('*** ', SP.SP)

In [ ]:

print('** ', SACCT_obj.jobs_summary.dtype)
print('** ', type(SACCT_obj.jobs_summary['User'][0]))
#
# my_ary = numpy.array(len(SACCT_obj.jobs_summary), dtype=SACCT_obj.jobs_summary.dtype)
# print('** ', my_ary.dtype)
#
for cl,tp in SACCT_obj.jobs_summary.dtype.descr:
    print('** ', cl, tp)
    
#
print('** ** ', SACCT_obj.jobs_summary['Group'].astype(str)[0])


In [ ]:
rep_cpu_lc = SACCT_obj.report_activecpus_jobs_layercake_and_CDFs(group_by='Partition', trendline=True)
#
# plot total CPUs in partition. Also, subtract CPUs associated with GPUs. For now, this is just
#. something we know. we will need to work harder to get it from data.
ax = rep_cpu_lc.axes[0]
ln = ax.lines[0]
#
x_max_cpus = numpy.array([ax.lines[0].get_xdata()[0], ax.lines[0].get_xdata()[-1]])
ax.plot( x_max_cpus, numpy.ones(2)*n_cpus_serc, ls='--', lw=3.,
       color='r', label='+GPUs')

# TODO: smarter waty to count GPU-CPUs in SINFO...
ax.plot( x_max_cpus, numpy.ones(2)*n_cpus_serc-(10*128 + 2*24 + 2*64), ls='--', lw=3.,
       color='m', label='CPUs')
#
# ax.plot( x_max_cpus, numpy.ones(2)*n_cpus_serc-(10*128 + 2*24 + 96*32), ls='--', lw=3.,
#        color='c', label='CPUs (old SERC)')

ax.legend(loc='upper left')

In [ ]:
print('** ', SACCT_obj.jobs_summary.dtype.names)

In [ ]:
# Active CPUs and quantiels for CPU Only jobs:
ix = SACCT_obj['NGPUs']==0
rep_cpu_CPU_jobs = SACCT_obj.report_activecpus_jobs_layercake_and_CDFs(group_by='Partition',
                                                jobs_summary=SACCT_obj[ix], trendline=True)
fg = rep_cpu_CPU_jobs
ax = rep_cpu_CPU_jobs.axes[0]
ln = ax.lines[0]
#
N_cpu_cpus = n_cpus_serc-(10*128 + 2*24 + 1*64)
#
x_max_cpus = numpy.array([ax.lines[0].get_xdata()[0], ax.lines[0].get_xdata()[-1]])
ax.plot( x_max_cpus, numpy.ones(2)*N_cpu_cpus, ls='--', lw=3.,
       color='m', label=f'CPUs: {N_cpu_cpus}')
fg.suptitle('CPU Jobs (GPUs excluded)', size=14)
ax.legend(loc=0)

## Wait times (CPU only jobs)


In [ ]:
numpy.sum(~numpy.isnan(SACCT_obj['End']))


In [ ]:
ix_wt_cpu = numpy.logical_and(SACCT_obj['NGPUs']==0, ~numpy.isnan(SACCT_obj['End']))
ix_wt_gpu = numpy.logical_and(SACCT_obj['NGPUs']>0, ~numpy.isnan(SACCT_obj['End']))
#
wait_times_cpu = 24.*(SACCT_obj['End'][ix_wt_cpu] - SACCT_obj['Start'][ix_wt_cpu])
wait_times_gpu = 24.*(SACCT_obj['End'][ix_wt_gpu] - SACCT_obj['Start'][ix_wt_gpu])
#
q_steps = [.5, .75, .9, .99]
qs_cpu = numpy.quantile(wait_times_cpu[~numpy.isnan(wait_times_cpu)], q_steps)
qs_gpu = numpy.quantile(wait_times_gpu[~numpy.isnan(wait_times_gpu)], q_steps)
#
print('CPUs: \n', qs_cpu)
print('GPUs: \n', qs_gpu)
#
fg = plt.figure(figsize=(12,10))
ax1 = fg.add_subplot(1,2,1)
ax2 = fg.add_subplot(1,2,2)
#
h1 = ax1.hist(wait_times_cpu, bins=100, cumulative=True, histtype='step', lw=3, density=True)
h2 = ax2.hist(wait_times_gpu, bins=100, cumulative=True, histtype='step', lw=3, density=True)
#
for t,q in zip(qs_cpu, q_steps):
    ax1.plot([t,t,0], [0,q,q], label=f'q_{int(100*q)}={t:.2f} hour')
for t,q in zip(qs_gpu, q_steps):
    ax2.plot([t,t,0], [0,q,q], label=f'q_{int(100*q)}={t:.2f} hour')

ax1.set_title('Wait times, CPU jobs', size=16)
ax2.set_title('Wait times, GPU jobs', size=16)
ax1.legend(loc=0)
ax2.legend(loc=0)
ax1.grid()
ax2.grid()
#


In [ ]:
#fg_cpuhl_acct = SACCT_obj.report_cpuhours_jobs_layercake_and_pie(group_by='Account')
fg_cpu_lc_acct = SACCT_obj.report_activecpus_jobs_layercake_and_CDFs(group_by='Account')

In [ ]:
# total CPU hours, by group:
#grp_total_cpuhours = numpy.zeros(len(numpy.unique(SACCT_obj['Account'])))
grp_total_cpuhours = []
SO = SACCT_obj
#
for grp in numpy.unique(SO['Account']):
    #if isinstance(grp, bytes):
    #    grp=grp.decode()
    #print(f'*** {grp}' )
    
    #ix = SO['Account']==grp.encode()
    ix = SO['Account']==grp
    grp_total_cpuhours += [[grp, numpy.sum(SO['NCPUS'][ix] * SO['Elapsed'][ix]) ]]
    #
for k,(nm, x)in enumerate( sorted(grp_total_cpuhours, key=lambda rw:rw[1])[::-1] ):
    print(f'[{k}] {nm}: {x:.2f} ')


In [ ]:
grp_total_cpuhours = []
#SO = SACCT_obj[SACCT_obj['NGPUs']>0]
SO = SACCT_obj
#
for grp in numpy.unique(SO['Account']):
    #if isinstance(grp, bytes):
    #    grp=grp.decode()
    #print(f'*** {grp}' )
    #ix = SO['Account']==grp.encode()
    ix = numpy.logical_and(SO['Account']==grp, SO['NGPUs']>0)
    grp_total_cpuhours += [[grp, numpy.sum(SO['NCPUS'][ix] * SO['Elapsed'][ix]),
                            numpy.sum(SO['NGPUs'][ix] * SO['Elapsed'][ix]) ]]
    #
print('** group: cpu-hours  gpu-hours')
for k,(nm, c, g)in enumerate( sorted(grp_total_cpuhours, key=lambda rw:rw[2])[::-1] ):
    if g==0.: continue
    print(f'[{k}] {nm}: {c:.2f}, {g:.2f}')

## CPU-hours distribution
This will be a sort of odd distribution, I think. The idea is to capture how much of our compute is executed on how many cpus or nodes, so we can be better informed as to what sort of HW to buy.

So the idea will be aggregate `CPUs x hours` in bins of `ncpus` or `nnodes`.

In [ ]:
print('** ', SACCT_obj.jobs_summary.dtype.names)
print('**\n**\n')
print(numpy.unique(SACCT_obj['NCPUS']))

In [ ]:
cpu_hours_dist   = numpy.zeros(len(numpy.unique(SACCT_obj['NCPUS'])), dtype=[('NCPUS', '<i8'),
                                                                ('hours', '<f8'), ('totalcpu', '<f8')])
cpu_hours_dist['NCPUS'] = numpy.unique(SACCT_obj['NCPUS'])

node_hours_dist = numpy.zeros(len(numpy.unique(SACCT_obj['NNodes'])), dtype=[('NNodes', '<i8'),
                                                                ('hours', '<f8'), ('totalcpu', '<f8')])
node_hours_dist['NNodes'] = numpy.unique(SACCT_obj['NNodes'])
#
#unique_ntasks = numpy.unique((SACCT_obj['NTasks'])[SACCT_obj['NTasks']>0])
unique_ntasks = numpy.unique((SACCT_obj['NTasks']))
tasks_hours_dist = numpy.zeros(len(unique_ntasks), dtype=[('NTasks', '<i8'),
                                                                ('hours', '<f8'), ('totalcpu', '<f8')])
tasks_hours_dist['NTasks'] = unique_ntasks
#
# Ok... probably should have started with a dict, then translated to an array, or use PANDAS. Or use a full
# range() of index values, so we can just use spatial indexing, but we're not. We'll just use dict() as 
# an index to assign aggregates.
#
# also, this will not be the fastest way to do this, but it should be fairly straight forward.
ix_cpus  = {n:k for k,n in enumerate(cpu_hours_dist['NCPUS'])}
ix_nodes = {n:k for k,n in enumerate(node_hours_dist['NNodes'])}
ix_tasks = {n:k for k,n in enumerate(tasks_hours_dist['NTasks'])}
for rw in SACCT_obj:
    #print('** ')
    this_cpuh = rw['Elapsed']*rw['NCPUS']
    for var,ix,cl in [(cpu_hours_dist, ix_cpus, 'NCPUS'), (node_hours_dist, ix_nodes, 'NNodes'),
                      (tasks_hours_dist, ix_tasks, 'NTasks')]:
        if numpy.isnan(rw[cl]):
            continue
        var['hours'][ix[int(rw[cl])]] += this_cpuh
        var['totalcpu'][ix[int(rw[cl])]] += rw['TotalCPU']
    #cpu_hours_dist['hours'][ix_cpus[int(rw['NCPUS'])]] += this_cpuh
    #node_hours_dist['hours'][ix_nodes[int(rw['NNodes'])]] += this_cpuh
    #tasks_hours_dist['hours'][ix_tasks[int(rw['NTasks'])]] += this_cpuh
    #
    # this is effectively a "jobwise" average. Or we just collect ['TotalCPU'] and divide at the end...
    #tasks_hours_dist['eff'][ix_tasks[int(rw['NTasks'])]] += rw['TotalCPU']/(['Elapsed']*rw['NCPUS'])
    #cpu_hours_dist['totalcpu'][ix_cpus[int(rw['NCPUS'])]] += rw['TotalCPU']
    #node_hours_dist['totalcpu'][ix_nodes[int(rw['NNodes'])]] += rw['TotalCPU']
    #tasks_hours_dist['totalcpu'][ix_tasks[int(rw['NTasks'])]] += rw['TotalCPU']
#
tasks_hours_dist = tasks_hours_dist[tasks_hours_dist['NTasks']>0]

cpu_hours_dist['hours'] = numpy.cumsum(cpu_hours_dist['hours'])
node_hours_dist['hours'] = numpy.cumsum(node_hours_dist['hours'])
tasks_hours_dist['hours'] = numpy.cumsum(tasks_hours_dist['hours'])





qs = numpy.array([.5, .75, .9])
fg = plt.figure(figsize=(14,6))
#
ax1 = fg.add_subplot(1,3,1)
ax2 = fg.add_subplot(1,3,2)
ax3 = fg.add_subplot(1,3,3)
#
axs = [ax1, ax2, ax3]
#
fg.suptitle('CPU-hours CDFs')
for ax,ttl, in zip(axs, ['Per NCPU', 'Per Node', 'Per Task']):
    ax.set_title(ttl)
    ax.grid()
    #
X = cpu_hours_dist['NCPUS']
Y = cpu_hours_dist['hours']
#quants = numpy.quantile(Y, qs)
quants = numpy.nanmax(Y)*qs
ax = ax1
ax.plot(X,Y)
ks = Y.searchsorted(quants)
#
ax.set_xlabel('NCPUs', size=16)
ax.set_ylabel('$CPUs \cdot hours$', size=16)
for k,(j,y) in enumerate(zip(Y.searchsorted(quants), quants)):
    x = numpy.mean(X[j-1:j+1])
    ax.plot([x,x,numpy.min(X)], [numpy.min(Y),y,y], ls='-', label=f'q: {qs[k]}={x}')
ax.legend(loc=0)
###########
#
X,Y = node_hours_dist['NNodes'], node_hours_dist['hours']
quants = numpy.nanmax(Y)*qs
ax = ax2
#
ax.plot(X,Y)
ax.set_xlabel('N_Nodes', size=16)
#ax.set_ylabel('$Nodes \cdot hours$', size=16)
for k,(j,y) in enumerate(zip(node_hours_dist['hours'].searchsorted(quants), quants)):
    x = numpy.mean(X[j-1:j+1])
    ax.plot([x,x,numpy.min(X)], [numpy.min(Y),y,y], ls='-', label=f'q: {qs[k]}={x}')
ax.legend(loc='lower right')
################
#
X,Y = tasks_hours_dist['NTasks'], tasks_hours_dist['hours']
quants = numpy.nanmax(Y)*qs
ax = ax3
#
ax.plot(X,Y)
ax.set_xlabel('N_Tasks', size=16)
#ax.set_ylabel('$Tasks \cdot hours$', size=16)
for k,(j,y) in enumerate(zip(Y.searchsorted(quants), quants)):
    x = numpy.mean(X[j-1:j+1])
    ax.plot([x,x,numpy.min(X)], [numpy.min(Y),y,y], ls='-', label=f'q: {qs[k]}={x}')
ax.legend(loc='lower right')

In [ ]:
for x_var in ['NCPUS', 'NNodes', 'NTasks']:
    
    SO = SACCT_obj[SACCT_obj[x_var]==1]
    cpudays_one_task = numpy.sum(SO['NCPUS']*SO['Elapsed'])
    cpudays_all      = numpy.sum(SACCT_obj['NCPUS']*SACCT_obj['Elapsed'])
    print(f'** {x_var}:: one: {24.*cpudays_one_task:.2f}, all: {24.*cpudays_all:.2f}: fract: {cpudays_one_task/cpudays_all:.2f}')

### CPU Efficiency distribution(s)


In [ ]:
fg = plt.figure(figsize=(8,6))
ax = plt.gca()
ax.grid()
#
eff = tasks_hours_dist['totalcpu']/tasks_hours_dist['hours']
#
ax.plot(cpu_hours_dist['NCPUS'][0:],    (cpu_hours_dist['totalcpu']/cpu_hours_dist['hours'])[0:],
        marker='o', ls='-', label='cpus')
ax.plot(node_hours_dist['NNodes'][0:], (node_hours_dist['totalcpu']/node_hours_dist['hours'])[0:],
        marker='o', ls='-', label='nodes')
ax.plot(tasks_hours_dist['NTasks'][0:], (tasks_hours_dist['totalcpu']/tasks_hours_dist['hours'])[0:],
        marker='o', ls='-', label='tasks')
ax.legend(loc=0)

### User stat table
- Generate a table of summary user stats

In [ ]:
#
cpuh_pie_user = hpc_lib.get_pie_slices(sum_data=SACCT_obj['Elapsed']*SACCT_obj['NCPUS'],
                                       slice_data=SACCT_obj['User'])
jobs_pie_user = hpc_lib.get_pie_slices(sum_data=SACCT_obj['Elapsed'], slice_data=SACCT_obj['User'])
#

In [ ]:
# This will be way too big to produce in notebooks, practically speaking at least..
# or maybe not, but we should do it last?
t0 = mpd.date2num(start_date)
t1 = mpd.date2num(end_date)
delim = chr(9)
delim = ';'
print('*** CPU-hours: ')
#print('**  Name,   cpu-hours,    job-hours,  last_job_start', )
print(delim.join(['Name', 'cpu-hours', 'job-hours', 'n_jobs', 'last_job_start', 'Group', 'Accounts', 'Partitions']))
jindex = {nm:k for k,nm in enumerate(jobs_pie_user['name'].astype(str))}
#print('** jindex: ', jindex)
for k, (nm,n) in enumerate(cpuh_pie_user[numpy.argsort(cpuh_pie_user['value'])[::-1]] ):
    if k>10: break
    #
    ix = SACCT_obj['User'].astype(type(nm)) == nm
    fg = plt.figure(figsize=(10,4))
    ax = fg.add_subplot(1,1,1)
    z = SACCT_obj.get_cpu_hours(jobs_summary=SACCT_obj[ix])
    ax.plot(z['time'], z['cpu_hours'], ls='-', marker='')
    ax.set_xlim(t0,t1)
    ax.grid()
    ax.set_xlabel('time $t$', size=16)
    ax.set_ylabel('cpu-hours (per day?)', size=16)
    #
    if isinstance(nm,bytes):
        nm = nm.decode()
    #
    ax.set_title(nm, size=16)
    #
    fg.canvas.draw()
    dt_epoch = hpc_lib.compute_mpd_epoch_dt(z['time'][0])
    lbls = [hpc_lib.simple_date_string(mpd.num2date(x + dt_epoch)) for x in ax.get_xticks()]
#     lbls = [hpc_lib.simple_date_string(mpd.num2date(float(hpc_lib.fix_to_ascii(str(s.get_value()))) + dt_epoch) )
#               for s in ax.get_xticklabels()]

    #
    #ax.set_xticklabels(lbls)
    # This should get rid of the FixedLocator warning? But I'm not sure it will...
    ticks_loc = ax.get_xticks().tolist()
    ax.xaxis.set_major_locator(mpl.ticker.FixedLocator(ticks_loc))
    ax.set_xticklabels(lbls)
    #
    rw_vals = [nm, n, jobs_pie_user['value'][jindex[nm]], numpy.sum(ix).astype(int),\
          (None if numpy.isnan(numpy.nanmax(numpy.nanmax(SACCT_obj['Start'][ix]))) else mpd.num2date(numpy.nanmax(SACCT_obj['Start'][ix])) ),\
           SACCT_obj['Group'][ix].astype(str)[0],\
                ','.join(numpy.unique(SACCT_obj['Account'][ix]).astype(str)),\
                ','.join(numpy.unique(SACCT_obj['Partition'][ix]).astype(str))]
    print(delim.join([str(x) for x in rw_vals]))
#     print(f"{nm.decode()}, {n}, {jobs_pie_user['value'][jindex[nm]]},\
#           {mpd.num2date(max(SACCT_obj['Start'][ix]))}, {SACCT_obj['Group'][ix].astype(str)[0]},\
#                 {delim.join(numpy.unique(SACCT_obj['Account'][ix]).astype(str))},\
#                 {delim.join(numpy.unique(SACCT_obj['Partition'][ix]).astype(str))}\
#                 ")
#
# print('*** Jobs-time:')
# print('**  Name,   n_jobs,   last_job_start')
# for nm,n in jobs_pie_user[numpy.argsort(jobs_pie_user['value'])[::-1]]:
#     ix = SACCT_obj['User'].astype(type(nm)) == nm
#     print(f"**  {nm.decode()}, {n}, {mpd.num2date(max(SACCT_obj['Start'][ix]))}, {SACCT_obj['Group'][ix][0]}")

In [ ]:
# DEBUG:
print('** ', ax)
print('** ', ax.get_xticklabels()[0].get_position()[0])
print('** ', ax.get_xticks())

In [ ]:
# Now, let's get a some reports for specific users, namely Lauren and Eric's former student(s) to estimate
#. requirements for their successors.
#
# Also, TODO: layer cake for active_cpus ?
# NOTE: for up and coming "how busy is the queue?" reporting, something like this:
# squeue -p serc --Format=jobid,jobarrayid,partition,username,state,timeused,timeleft,allocnodes,numnodes,numcpus


In [ ]:
cpuh_jobs = SACCT_obj.get_cpu_hours(bin_size=1., n_points=5000)
cpuh_layers = SACCT_obj.get_cpu_hours_layer_cake(bin_size=1.)

fg = plt.figure(figsize=(20,16))
ax1 = fg.add_subplot(2,2,1)
ax2 = fg.add_subplot(2,2,2)
ax3 = fg.add_subplot(2,2,3)
ax4 = fg.add_subplot(2,2,4)
ax1.grid()
ax2.grid()
#
ax1.set_title('CPU Hours (per day)', size=16)
ax2.set_title('Jobs (per day?)', size=16)
#
cpuh = cpuh_layers['cpu_hours']
jobs = cpuh_layers['jobs']
T = cpuh['time']

print('*** ', cpuh.dtype)
print('*** ', cpuh['time'][0:10])

#
z_cpuh = hpc_lib.plot_layer_cake(data=cpuh, layers=cpuh.dtype.names[1:], time_col='time', ax=ax1)
z_jobs = hpc_lib.plot_layer_cake(data=jobs, layers=cpuh.dtype.names[1:], time_col='time', ax=ax2)
#
# pi charts. left: cpu-hours, right job-time
# NOTE: these times will be in units of "days". Not important to convert them unless we need to quantify.
pi_cpuh = hpc_lib.get_pie_slices(sum_data=SACCT_obj['Elapsed']*SACCT_obj['NCPUS'], slice_data=SACCT_obj['Account'])
pi_cpuh_lbls = pi_cpuh['name'].astype(str)
pi_cpuh_vls  = pi_cpuh['value'].astype(str)

pi_jobs = hpc_lib.get_pie_slices(sum_data=SACCT_obj['Elapsed'], slice_data=SACCT_obj['Account'])
pi_jobs_lbls = pi_jobs['name'].astype(str)
pi_jobs_vls  = pi_jobs['value'].astype(str)
#
ax3.pie(pi_cpuh_vls, labels=pi_cpuh_lbls)
ax4.pie(pi_jobs_vls, labels=pi_jobs_lbls)
#
ax1.legend(loc=0)
ax2.legend(loc=0)
#
# fg.canvas.draw()
# for ax in (ax1, ax2):
#     lbls = [hpc_lib.simple_date_string(mpd.num2date(float(hpc_lib.fix_to_ascii(str(s.get_text()))) + SACCT_obj.dt_mpd_epoch ) ) 
#              for s in ax.get_xticklabels()]
#     ax.set_xticklabels(lbls)
#fg.canvas.draw()
#lbls = [hpc_lib.simple_date_string(mpd.num2date(float(hpc_lib.fix_to_ascii(str(s.get_text())))) ) 
#         for s in ax1.get_xticklabels()]
#ax1.set_xticklabels(lbls)


In [ ]:
print('index, PI_group, CPUhours')
ix = numpy.argsort([pi_cpuh['value']])

pi_cpuh.sort(order='value')

for k, (usr,cpuh) in enumerate(pi_cpuh[::-1]):
    if hasattr(usr, 'decode'):
        usr = usr.dedoce(())
    print(f'{k}, {usr}, {cpuh*24.:.1f} ', )

# for k, (pi,hrs) in enumerate(pi_cpuh[::-1]):
#     print(f'{k}, {pi.decode()}, {pi_cpuh}')
#CDF = numpy.cumsum(pi_cpuh['value'])

fg = plt.figure(figsize=(10,8))
ax = fg.add_subplot(1,1,1)
#
ax.plot(numpy.arange(len(pi_cpuh)), 24.*numpy.cumsum(pi_cpuh['value'][::-1])   )

In [ ]:
# CPU Hours CDF
fg = plt.figure(figsize=(16,8))
ax1 = fg.add_subplot(1,2,1)
ax2 = fg.add_subplot(1,2,2)
h  = ax1.hist(pi_cpuh['value'], bins=len(pi_cpuh), cumulative=True, density=True, histtype='step', lw=3)
h2 = ax2.hist(pi_cpuh['value'], bins=len(pi_cpuh), cumulative=False, density=False, histtype='step', lw=3)
ax1.grid()
ax1.set_title('CDF of CPU Hours, by PI group', size=16)
ax1.set_xlabel('CPU Hours', size=16)

ax2.set_yscale('log')


### SERC GPU activity

### GPUs:
For now, hijack the SACCT.get_active_cpus_layer_cake() function, but force the "CPUs" column to use GPUs.

In [ ]:
gpu_layers = SACCT_obj.get_active_cpus_layer_cake(layer_field='Account', NCPUs=SACCT_obj.get_NGPUs())

In [ ]:
print(gpu_layers.keys())

In [ ]:
# NOTE: here, N_cpu is GPUs, since GPU s were substituted into the CPU field to get the layers.
pi_gpu_grps = [s for s in gpu_layers['N_cpu'].dtype.names[1:]]
#print(f'** {pi_gpu_lbls}' )
#
NGPU = SACCT_obj.get_NGPUs()
#
#pi_gpu_vals = numpy.zeros(len(pi_gpu_lbls))
pi_gpu_vals = []
pi_gpu_lbls = []
for k,g in enumerate(pi_gpu_grps):
    ix = SACCT_obj.jobs_summary['Group'].astype(str)==g
    #
    n_gpus = numpy.sum(SACCT_obj.jobs_summary['Elapsed'][ix] * NGPU[ix])
    if n_gpus <= 0.:
        continue
    #
    pi_gpu_vals += [n_gpus]
    pi_gpu_lbls += [f'{g}: {pi_gpu_vals[-1]:.1f}']
#
#print('** vals: ', pi_gpu_vals)
#pi_gpu_lbls = [f'{s}: {v:.1f}' for s,v in zip(pi_gpu_lbls, pi_gpu_vals) ]


fg = plt.figure(figsize=(16,14))
ax1 = fg.add_subplot(2,1,1)
ax2 = fg.add_subplot(2,2,3)
ax3 = fg.add_subplot(2,2,4)
ax1.grid()
#ax2.grid()
ax3.grid()
#
hpc_lib.plot_layer_cake(gpu_layers['N_cpu'], ax=ax1)
z_gpus = ax1.lines[-1].get_ydata()
x_gpus = ax1.lines[-1].get_xdata()
#
qs = [.5, .75, .9]
qs_gpu = numpy.quantile(z_gpus, qs)
#
# and a trend analysis:
A = numpy.vstack([x_gpus**n for n in range(2)]).T
p = numpy.linalg.lstsq(A, z_gpus)[0]
A = A[0::len(A)-1]
ax1.plot(A[:,1], numpy.dot(A,p), ls='--', lw=3, label=f'trend: $b={p[1]:.2f}')
#
print('*** keys(): ', gpu_layers.keys())
ax2.pie(pi_gpu_vals, labels=pi_gpu_lbls) 
ax2.legend(loc='upper right')
#
hh_cpus = ax3.hist(z_gpus, bins=100, cumulative=True, density=True, histtype='step', lw=3.)
for x,y in zip(qs_gpu, qs):
    #ax3.plot([0., qs_cpus[-1], qs_cpus[-1]], [qs[-1], qs[-1], 0.], ls='--', color='r', lw=2. )
    ax3.plot([0., x, x], [y, y, 0.], ls='--', lw=2., label=f'{y*100.}th %: {x:.0f} gpus' )

ax1.set_title('Active GPUs', size=16)
ax3.legend(loc=0)


In [ ]:
fg = plt.figure(figsize=(8,6))
ax1 = fg.add_subplot(1,1,1)
#
hh_cpus = ax1.hist(z_gpus, bins=100, cumulative=True, density=True, histtype='step', lw=3.)
for x,y in zip(qs_gpu, qs):
    #ax3.plot([0., qs_cpus[-1], qs_cpus[-1]], [qs[-1], qs[-1], 0.], ls='--', color='r', lw=2. )
    ax1.plot([0., x, x], [y, y, 0.], ls='--', lw=2., label=f'{y*100.}th %: {x:.0f} gpus' )
    
ax1.grid()
ax1.legend(loc=0)

In [ ]:
SACCT_obj.dtype.names

In [ ]:
# Big GPU groups:
#ix_gpu_user_groups = numpy.argsort()
# print('** ', pi_gpu_grps, len(pi_gpu_grps))
# print('** ', pi_gpu_vals, len(pi_gpu_vals))
#
#ZZ = numpy.array([pi_gpu_grps, pi_gpu_vals]).T
gpu_users = []
for grp in pi_gpu_grps:
    #print(f'** {grp}, {dt}*24.')
    ix = SACCT_obj['Account'].astype(str) == grp
    S = SACCT_obj[ix]
    gpu_hours = 24.*numpy.sum(S['Elapsed'] * S['NGPUs'])
    if gpu_hours == 0.:
        continue
    #
    gpu_users += [[grp, gpu_hours]]
    #print(f'** {grp}: {gpu_hours}')
gpu_users.sort(key=lambda rw: rw[1])

for g,dt in gpu_users[::-1]:
    print(f'{g}: {dt:.2f}')
    
    



In [ ]:
cpu_jobs_serc = SACCT_obj.active_jobs_cpu()


In [ ]:
serc_cpu_qs = numpy.quantile(cpu_jobs_serc['N_cpu'], [.5, .75, .9])
print('** qs: ', serc_cpu_qs)

In [ ]:
print('** ', numpy.unique(SACCT_obj['NGPUs']))
#
# for now, just manually add a max_mem value, in raw format...:
# ...and I'm going to make it a little bigger, to avoid GB vs Gb (or whatver...) issues.
#max_mem = 1024.*(1024.**3.)*1.1
max_mem = 1024.*GB
#
ix_gpu = SACCT_obj['NGPUs']>0
ix_gpu = numpy.logical_and(ix_gpu, SACCT_obj['ReqMem']<=max_mem)
#
ix_cpu = numpy.invert(ix_gpu)
#
# Diagnostic sequences and figures:
mpG = numpy.zeros(len(SACCT_obj.jobs_summary))
mpG[ix_gpu] = SACCT_obj['ReqMem'][ix_gpu] / SACCT_obj['NGPUs'][ix_gpu]
mpG /= GB
#
ix_bigguns = numpy.argsort(mpG)
print(f'** Bigguns indices: {ix_bigguns[-5:]}')
print(f'** Bigguns: {mpG[ix_bigguns[:-5]]}')
k_max = ix_bigguns[-1]
print('** ', mpG)
print('** ', SACCT_obj[-1])

######
fg = plt.figure(figsize=(8,6))
ax = plt.gca()
ax.plot(numpy.arange(len(mpG)), mpG)
#ax.plot(SACCT_obj['Start'][ix_gpu], mpG[ix_gpu], ls='-', lw=2.)
ax.grid()
ax.set_xlabel('job sequence')
ax.set_ylabel('mem/GPU')

# Quantiles plot:
ave_len=100
mpg = numpy.array([x for x in mpG if x>0])
gpu_quants = numpy.array([numpy.quantile(mpg[k-ave_len:k], [.5, .75, .95]) for k in range(ave_len, len(mpg))])

fg = plt.figure(figsize=(8,6))
ax = plt.gca()
low, med, hi = (gpu_quants[:,k] for k in numpy.arange(gpu_quants.shape[1]))
X = numpy.arange(len(low))

l_lo, = plt.plot(X,low, ls='-', lw=1)
ax.plot(X, med, ls='-', lw=3)
l_hi, = plt.plot(X, hi, ls='-', lw=1)

ax.fill_between(X, low, med, color=l_lo.get_color(), alpha=.2)
ax.fill_between(X, med, hi, color=l_hi.get_color(), alpha=.2)
ax.grid()



In [ ]:
ave_len=25
mpg = numpy.array([x for x in mpG if x>0])
gpu_quants = numpy.array([numpy.quantile(mpg[k-ave_len:k], [.5, .75, .9]) for k in range(ave_len, len(mpg))])
print('*** ', gpu_quants.shape)

In [ ]:
#
sacct_gpus = SACCT_obj.jobs_summary[ix_gpu]
#mem_per_gpu= SACCT_server[]
#
print('*** ', len(SACCT_obj.jobs_summary), len(sacct_gpus))
#
mem_per_gpu = (sacct_gpus['ReqMem']/sacct_gpus['NGPUs'])/GB
mem_per_cpu = (SACCT_obj['ReqMem'][ix_cpu] / SACCT_obj['NCPUS'][ix_cpu])/GB

rss_per_gpu = (sacct_gpus['MaxRSS']/sacct_gpus['NGPUs'])/GB

#req_mem = SACCT_obj['ReqMem']/GB
#
fg = plt.figure(figsize=(10,8))
ax1 = fg.add_subplot(1,1,1)
#ax2 = fg.add_subplot(1,2,2)
#
hh_rss = ax1.hist(rss_per_gpu, bins=20, label='MaxRSS')
hh_gpu = ax1.hist(mem_per_gpu, bins=20, histtype='step', label='ReqMem')

##hh_cpu = ax2.hist(mem_per_cpu, bins=20)
#hh_cpu = ax2.hist( (SACCT_obj['ReqMem']/(1024**3)), bins=2)
#
#ax1.set_yscale('log')
#ax2.set_yscale('log')

ax1.grid()
#ax2.grid()
#fg.suptitle('Requested Memory', size=16)
ax1.set_title('ReqMem/GPU')
ax1.set_xlabel('Memory per GPU', size=16)
ax1.set_ylabel('Num. reqeusts', size=16)
ax1.legend(loc=0)

### ReqMem/GPU, Tabular output

In [ ]:
# Requested memory for GPU jobs, Tabular output.
#
# Tabular output:
# Numbers of GPUs-hrs, per GPU/mem
#
#gpu_hrs_mem = {(k+1)*128:0 for k in range(8)}
gpu_hrs_mem = numpy.array([[(k+1)*128, 0., 0., 0., 0.] for k in range(8)])

#mpgs = numpy.array(list(gpu_hrs_mem.keys()))
mpgs = gpu_hrs_mem[:,0]
#print('** ', mpgs)

print('** gpu_hrs_mem, Initialized:\n', gpu_hrs_mem)
# for kk, rw in enumerate(sacct_gpus):
#     N = rw['NGPUs']
#     Z = rw['ReqMem']/(GB*N)
#     k = numpy.searchsorted( mpgs, Z )
#     print(f'** mpg: {Z}, k: {k}')
#     if kk>10: break
    
Ks = numpy.searchsorted( mpgs, sacct_gpus['ReqMem']/(GB*sacct_gpus['NGPUs']))
total_gpus = numpy.sum(sacct_gpus['NGPUs'])
total_hours = numpy.sum(sacct_gpus['NGPUs']*sacct_gpus['Elapsed']*24.)
print('*** ', Ks[0:10])
for k,rw in zip(Ks,sacct_gpus):
    gpu_hrs_mem[k][1] += rw['NGPUs']
    gpu_hrs_mem[k][2] += rw['NGPUs']*rw['Elapsed']*24.   # in hrs.
#
gpu_hrs_mem[:,3] = (100.*gpu_hrs_mem[:,1]/total_gpus)
gpu_hrs_mem[:,4] = (100.*gpu_hrs_mem[:,2]/total_hours)

print('** gpu_hrs_mem, completed?')
for m,n,t,pn,pt in gpu_hrs_mem:
    print(f'{m:.0f}  {n:.0f} {t:.2f} {pn:.2f}  {pt:.2f}')
    #print(f'{m:f}  {n:f} {t:f} {pn:f}  {pt:f}')

print('\n\nReqMem/GPU time distributions')
print('m/gpu  n_gpus     gpu-time      pct_ngpu    pct_time')
for m,n,t,pn,pt in gpu_hrs_mem:
    
    mm  = ('{: <6}'.format(m))
    nn  = ('{: >7}'.format(f'{n:.2f}'))
    tt  = ('{: >10}'.format(f'{t:.2f}'))
    ppn = ('{: >10}'.format(f'{pn:.2f}'))
    ppt = ('{: >10}'.format(f'{pt:.2f}'))
    
    print(f'{mm}  {nn} {tt} {ppn}%  {ppt}%')

    
 #   '{: <5}'.format(string)

### MaxRSS/GPU, Tabular output
- Copy ReqMem/GPU above

In [ ]:
print('** ', sacct_gpus['MaxRSS'][0:10])
print('** ', sacct_gpus['ReqMem'][0:10])

In [ ]:
gpu_hrs_mem = numpy.array([[(k+1)*128, 0., 0., 0., 0.] for k in range(8)])
#
# "memory per gpu"'s
mpgs = gpu_hrs_mem[:,0]
#
print('** gpu_hrs_mem, Initialized:\n', gpu_hrs_mem)
#
Ks = numpy.searchsorted( mpgs, sacct_gpus['MaxRSS']/(GB*sacct_gpus['NGPUs']))
total_gpus = numpy.sum(sacct_gpus['NGPUs'])
total_hours = numpy.sum(sacct_gpus['NGPUs']*sacct_gpus['Elapsed']*24.)
print('*** ', Ks[0:10])
for k,rw in zip(Ks,sacct_gpus):
    if k>=len(gpu_hrs_mem) or numpy.isnan(rw['MaxRSS']):
        continue
    #
    try:
        gpu_hrs_mem[k][1] += rw['NGPUs']
        gpu_hrs_mem[k][2] += rw['NGPUs']*rw['Elapsed']*24.   # in hrs.
    except:
        print(f'** EXCEPTION: MaxRSS={rw["MaxRSS"]}, k={k}')
        raise Exception()
#
gpu_hrs_mem[:,3] = (100.*gpu_hrs_mem[:,1]/total_gpus)
gpu_hrs_mem[:,4] = (100.*gpu_hrs_mem[:,2]/total_hours)

print('** gpu_hrs_mem, completed?')
for m,n,t,pn,pt in gpu_hrs_mem:
    print(f'{m:.0f}  {n:.0f} {t:.2f} {pn:.2f}  {pt:.2f}')
    #print(f'{m:f}  {n:f} {t:f} {pn:f}  {pt:f}')


print('\n\nMaxRSS/GPU time allocations')
print('m/gpu   n_gpus    gpu-time       pct_ngpu   pct_time')
for m,n,t,pn,pt in gpu_hrs_mem:
    
    mm  = ('{: <6}'.format(m))
    nn  = ('{: >7}'.format(f'{n:.2f}'))
    tt  = ('{: >10}'.format(f'{t:.2f}'))
    ppn = ('{: >10}'.format(f'{pn:.2f}'))
    ppt = ('{: >10}'.format(f'{pt:.2f}'))
    
    print(f'{mm}  {nn} {tt} {ppn}%  {ppt}%')

### GPU Jobs: ReqMem and MaxRSS per GPU and per Node

In [ ]:
print('** ', numpy.unique(SACCT_obj['NGPUs']))
#ix_gpu = SACCT_obj['NGPUs']>0
#ix_cpu = numpy.invert(ix_gpu)
#
sacct_gpus = SACCT_obj.jobs_summary[ix_gpu]
ix_cpu = numpy.invert(ix_gpu)
#mem_per_gpu= SACCT_server[]
#
print('*** ', len(SACCT_obj.jobs_summary), len(sacct_gpus))
#
#max_rss_gpu = sacct_gpus['MaxVMSize']/GB
max_rss_gpu = sacct_gpus['MaxRSS']/GB
req_mem_gpu = sacct_gpus['ReqMem']/GB
#
rss_per_gpu  = max_rss_gpu/sacct_gpus['NGPUs']
req_mem_per_gpu = req_mem_gpu/sacct_gpus['NGPUs']
#mem_per_cpu = (SACCT_obj['MaxRSS'][ix_cpu] / SACCT_obj['NCPUS'][ix_cpu])
#
# quantiles:
qs = [.5, .75, .90, .95, .99]
#
qs_rss_per_gpu  = numpy.quantile(rss_per_gpu[numpy.invert(numpy.isnan(rss_per_gpu))], qs)
qs_rss          = numpy.quantile(max_rss_gpu[numpy.invert(numpy.isnan(max_rss_gpu))], qs)
#
# Requested Memory per gpu and per node (for GPU jobs)
qs_rm_per_gpu  = numpy.quantile(req_mem_per_gpu[numpy.invert(numpy.isnan(req_mem_per_gpu))], qs)
qs_rm_per_node = numpy.quantile(req_mem_gpu[numpy.invert(numpy.isnan(req_mem_gpu))], qs)

print(f'** Quantiles: ({qs})')
print(f'rss/gpu: {qs_rss_per_gpu}')
print(f'rss/node: {qs_rss}')
print(f'req_mem/gpu: {qs_rm_per_gpu}')
print(f'req_mem/node: {qs_rm_per_node}')
#
fg = plt.figure(figsize=(10,12))
ax1 = fg.add_subplot(2,2,1)
ax2 = fg.add_subplot(2,2,2)
ax3 = fg.add_subplot(2,2,3)
ax4 = fg.add_subplot(2,2,4)
#
hh_gpu      = ax1.hist(rss_per_gpu, bins=20, density=True, label='MaxRSS')
hh_rmem_gpu = ax1.hist(req_mem_per_gpu, bins=20, density=True, label='ReqMem', histtype='step')
#
hh_gpu      = ax3.hist(rss_per_gpu, bins=20, cumulative=True, density=True, label='MaxRSS')
hh_rmem_gpu = ax3.hist(req_mem_per_gpu, bins=20, cumulative=True, histtype='step', density=True, label='ReqMem')
for x,y in zip(qs_rss_per_gpu, qs):
    ax3.plot([0., x, x], [y, y, 0.], ls='--', lw=2., label=f'{y*100.}th %: {x:.2f} GB/GPU' )
# for x,y in zip(qs_rm_per_gpu, qs):
#     ax3.plot([0., x, x], [y, y, 0.], ls='-.', lw=2., label=f'm{y*100.}th %: {x:.2f} GB/GPU' )


ax1.set_yscale('log')
ax2.set_yscale('log')


#hh_cpu = ax2.hist(mem_per_cpu, bins=20)
hh_cpu     = ax2.hist(max_rss_gpu, bins=20, density=True, label='MaxRSS')
hh_req_mem = ax2.hist(req_mem_gpu, bins=20, histtype='step', density=True, label='ReqMem')
#
hh_cpu     = ax4.hist(max_rss_gpu, bins=20, cumulative=True, density=True, label='MaxRSS')
hh_req_mem = ax4.hist(req_mem_gpu, bins=20, histtype='step', cumulative=True, density=True, label='ReqMem')
for x,y in zip(qs_rss, qs):
    ax4.plot([0., x, x], [y, y, 0.], ls='--', lw=2., label=f'{y*100.}th %: {x:.2f} GB/node' )
# for x,y in zip(qs_rm_per_node, qs):
#     ax4.plot([0., x, x], [y, y, 0.], ls='-.', lw=2., label=f'm{y*100.}th %: {x:.2f} GB/node' )


#

ax1.set_title('MaxRSS,ReqMem, per-gpu')
ax2.set_title('MaxRSS, ReqMem, per-node')
#
#ax3.set_title('RSS,ReqMem, per-cpu')
#ax4.set_title('MaxRSS,ReqMem, per-node (CPU)')

for ax in [ax1, ax2, ax3, ax4]:
    ax.legend(loc=0)
    ax.grid()
fg.suptitle('MaxRSS, ReqMem for GPU Jobs', size=16)

In [ ]:
print('** ', req_mem_per_gpu[0:10])

In [ ]:
gpu_wait_times = sacct_gpus['Start'] - sacct_gpus['Submit']

fg = plt.figure(figsize=(10,8))
ax1 = fg.add_subplot(1,1,1)

#hw = ax1.hist(gpu_wait_times, bins=10, histtype='step', density=True)
do_log=False
for m in numpy.arange(1,8):
    M0 = m*128.
    M1 = (m+1)*128
    #
    ix = numpy.logical_and(req_mem_per_gpu>M0, req_mem_per_gpu<=M1)
    hw = ax1.hist(24.*gpu_wait_times[ix], bins=20, histtype='step', label=M1, density=True, lw=3,log=do_log)
    

ax1.grid()
ax1.legend(loc=0)
ax1.set_title('GPU Wait times, for mem/GPU')
ax1.set_ylabel('Number of jobs')
ax1.set_xlabel('Wait time (days?)')




### Some Seasonality reports:
NOTE: Some reports being moved to hpc_reports module.

In [ ]:
cpu_seasonality = SACCT_obj.active_cpu_jobs_per_day_hour_report(qs=[.45, .5, .9], periodic_projection='polar')

cpu_seasonality = SACCT_obj.active_cpu_jobs_per_day_hour_report(qs=[.45, .5, .9], periodic_projection=None)

In [ ]:
# SACCT_rep = hpc_reports.SACCT_report_handler(SACCT_obj=SACCT_obj, Short_title='SERC, 2023-4',
#                                 Full_title="SERC HPC Analytics, April 2023", out_path='output/SERC_202304',
#                                             )

In [ ]:
#zz = SACCT_rep.cpu_hourly_activity_report()

t_end = SACCT_obj['End']
ix = numpy.isnan(t_end)
print('** ', numpy.sum(numpy.invert(ix)))

##  A Group report:
- specify group; all partitions.
- This is nominally (a version of) a standard report we might run for a PI group -- or at least this is how we produce the SACCT_obj for that group).

In [ ]:
# A bunch of group reports!
#  Well, a prototpy of that anaywayt.
#
# provide a list of valid PI_Groups (SACCT "Group" or possibly "Account" (which is the. same on Sherlock))

PI_Groups=['pi2', 'pi2', ...]
#
#['gorle', 'lou', 'oneillm', 'ramr', 'aditis2', 'jaramilo', 'horne']
cpu_hour_totals={}
#
for k_grp, grp_group in enumerate(PI_Groups):
    #grp_group='oneillm'
    #break
#     if k_grp >= 5 and False:
#         print('** BREAKing...')
#         break
    #
    break
    #n_cpus=10
    print(f'** Group: {grp_group}')
    #continue
    #
    grp_partition=None
    verbose=False
    SACCT_obj_grp = hpc_lib.SACCT_data_direct(group=grp_group, partition=grp_partition, start_date=str(start_date),
                                             n_cpu=os.environ['SLURM_CPUS_ON_NODE'], verbose=False, delim=delim_sacct,
                                             end_date=str(end_date), keep_raw_data=False)
    
    if len(SACCT_obj_grp)==0:
        print(f'*** *** Group: {grp_group} has no data')
        print(f'*** ***   Total CPU-hours[grp_group]: 0')
        cpu_hour_totals[grp_group]=0.
        #
        continue
    cpuh_total = numpy.sum(SACCT_obj_grp["Elapsed"]*SACCT_obj_grp["NCPUS"])
    cpu_hour_totals[grp_group]=cpuh_total
    print(f'** TOTAL CPU-hours [{grp_group}]: {cpuh_total:.2f}')
    #continue

    # Active CPUs layercake:
    try:
        fg_cpu_lc_acct = SACCT_obj_grp.report_activecpus_jobs_layercake_and_CDFs(group_by='Partition',
                                                                                 ave_N_layers=15,
                                                                                 ave_len_days=5.)

        ax = fg_cpu_lc_acct.axes[0]
        ln_ = ax.lines[0]
        X = ln_.get_xdata()
        #
        ax.plot(X[0::(len(X)-1)], (12*24)*numpy.ones(2), ls='--', lw=3)
        ax.set_title(f'Group: {grp_group}')
    except:
        print(f'*** EXCEPTION: activecpus_jobs_layercake_and_CDFS({grp_group}) failed')

    ####################################
    #


#     cpu_seasonality_group = SACCT_obj_grp.active_cpu_jobs_per_day_hour_report(qs=[.45, .5, .9],
#                                                                               periodic_projection='polar')
    
    
    #
    # plot_pie(sum_data=self['Elapsed']*self['NCPUS'], slice_data=self[group_by], ax=ax3, 
    #wedgeprops=wedgeprops, autopct=autopct, slice_names=cpuh.dtype.names[1:])

    fg = plt.figure(figsize=(8,6))
    ax1 = fg.add_subplot(1,1,1)
    users_cpuh = hpc_lib.plot_pie(sum_data=SACCT_obj_grp['Elapsed']*SACCT_obj_grp['NCPUS'],
    slice_data=SACCT_obj_grp['User'], ax=ax1)
    ax1.set_title(f'Group: {grp_group}: By User')

    fg = plt.figure(figsize=(8,6))
    ax = fg.add_subplot(1,1,1)
    #
    users_cpuh = hpc_lib.plot_pie(sum_data=SACCT_obj_grp['Elapsed']*SACCT_obj_grp['NCPUS'],
    slice_data=SACCT_obj_grp['Partition'], ax=ax)
    ax.set_title(f'Group: {grp_group}: By Partition')

    #################################
    #
#     fg = plt.figure(figsize=(10,6))
#     ax = fg.add_subplot(1,1,1)
#     #
#     # users layercake:P
#     users_apc_lc, users_jobs_lc = SACCT_obj_grp.get_active_cpus_layer_cake(layer_field='User').values()
#     #print('*** dtype: ', users_apc_lc.dtype.names)
#     zz = hpc_lib.plot_layer_cake(data=users_apc_lc, ax=ax, ave_len=15)
#     ax.set_title('Group Users layercake')
#     ax.legend(loc=0)
#     ax.grid()
#     ax.set_ylabel('Active CPUs')
    
#     N_users = len(users_apc_lc.dtype.names)-1
#     #ax.plot(zz[0::(len(zz)-1),0], numpy.ones(2)*500*N_users, ls='--', lw=3., color='r'   )

#     # to get individual user plots, for now, just do it the stupid way and de-layercake the output.
    

### Job size distribuitons
#### And other criteria for GCP allocations

In [ ]:

groups = PI_Groups

n_g = len(groups)
fg = plt.figure(figsize=(10, 6*n_g))
#fg2 = plt.figure(figsize=(10, 4*n_g))

print('** ')
#axes = []
for k,grp in enumerate(groups):
    print(f'** {grp}')
    if k>=0: break
    # not sure this index is being created correctly:
    ix = numpy.array([(g.decode() if isinstance(g,bytes) else g)==grp for g in SACCT_obj.jobs_summary['Account']])
    print('** ix: ', ix[0:10])
    jobs = SACCT_obj.jobs_summary[ix]
    #jobs = SACCT_obj.jobs_summary[(SACCT_obj.jobs_summary['Group']==grp)]
    
    ax1 = fg.add_subplot(n_g, 2, 2*k + 1)
    ax2 = fg.add_subplot(n_g, 2, 2*k + 2)
    ax1.set_title(f'Group: {grp}: Nodes')
    ax2.set_title(f'Group: {grp}: NCPUs')
    #
    ax1.grid()
    ax2.grid()
    #
    nbins=30
    h1 = ax1.hist(jobs['NNodes'], bins=nbins)
    h2 = ax2.hist(jobs['NCPUS'], bins=nbins)
    #
    ax1.set_xlabel('NNodes', size=16)
    ax2.set_xlabel('NCPUs', size=16)
    # User LayerCake:
    #
    # TODO: integrate into fg ? or separate 1 fg per group?
    #ax3 = fg2.add_subplot(n_g, 1, k+1)
    zz = SACCT_obj.report_activecpus_jobs_layercake_and_CDFs(group_by='User', jobs_summary=jobs)
    #ax3.set_title(f'Group: {grp} Layer-Cake, by user')
    #ax3.set_xlabel('time', size=16)
    #ax2.set_ylabel('Active CPUs', size=16)
    plt.gcf().suptitle(f'Group: {grp}')
    

## MaxVMSize (or MaxRSS) analysis for CPUs
- Turns out that MaxRSS reports the "largest" memory allocation for a job, which is not necessarily (???) the actual memory required.
- It seems that MaxVMSize is a more accurate (it tents to be bigger anyway...) measure of the actual memory required, and may be sum_{over steps?} (RSS).
- So the concern is that the MaxRSS analysis looks REALLY low for MPI jobs, and variuos forums report jobs failing with OoM when MaxRSS shows loads of available memory (like 10% allocated memory)
- One example given, suggesting that MaxRSS should be a valid measure of used memory, and predictor of whether a job will fail with OoM, disccusses dynamically allocated arrays (in Fortran), where `X` memory is allocated, then `x` memory is filled. The authors suggest that MaxRSS will show `x`, maybe MaxVMSize will show `X` ?
- Alternately, MaxVMSize shows the sum of the RSS for all the job steps? But nominally, should not the memory for those job steps be wiped clear?
- In any case, for now -- for this analysis, we will use MaxVMSize, since it is a little bit bigger, and so seems to be providing better resolution (of some sort?) of the resource use. AT this time, both analyses lead to similar conclusions with respect to how to use Sherlock resources and which new resources to purchase.
- On the other hand... `MaxVMSize` sometimes produces values (MUCH!) larger than the system memory. I think `MaxVMSize` allows for virtual memory constructs? Or Declarations of WAY too much memory, so long as it does not get filled?
- But Stephane suggests that `MaxRSS` is still the better choice -- that it reports "resident" memory, where `MaxVMSize` can report swap or linked libraries (which might not be resident. See:

    https://www.baeldung.com/linux/resident-set-vs-virtual-memory-size

In [ ]:
# print('** ', len((SACCT_obj['MaxRSS'].astype('>f8')/SACCT_obj['NCPUS'].astype('>f8'))))
# print('** ', (SACCT_obj['MaxRSS'].astype('>f8')/SACCT_obj['NCPUS'].astype('>f8'))[0:10])

In [ ]:
#ix1 = numpy.logical_and(SACCT_obj['NNodes']==1, SACCT_obj['State'].astype(str)=='COMPLETED')
ix1 = numpy.logical_and(SACCT_obj['NTasks']==1, SACCT_obj['State'].astype(str)=='COMPLETED')
#ix_mpi = numpy.logical_and(SACCT_obj['NTasks']>1, SACCT_obj['State'].astype(str)=='COMPLETED')
ix_mpi = numpy.logical_and(SACCT_obj['NNodes']>1, SACCT_obj['State'].astype(str)=='COMPLETED')
ix_gpu = (SACCT_obj['NGPUs']>1)

#
#MemVar = 'MaxVMSize'
MemVar = 'MaxRSS'
# maxrss/cpu and maxrss/node
# Let's see what (some of?) these look like when we use MaxVMSize instead...
max_rss_per_cpu_single = (1./GB)*(SACCT_obj[MemVar].astype('>f8')/SACCT_obj['NCPUS'].astype('>f8'))[ix1]
#max_rss_per_cpu_single = (1./GB)*(SACCT_obj['MaxVMSize'].astype('>f8')/SACCT_obj['NCPUS'].astype('>f8'))[ix1]
max_rss_per_cpu_single = max_rss_per_cpu_single[numpy.invert(numpy.isnan(max_rss_per_cpu_single).astype(bool))]
#
#
max_rss_per_node_single = (1./GB)*(SACCT_obj[MemVar].astype('>f8'))[ix1]
#max_rss_per_node_single = (1./GB)*(SACCT_obj['MaxVMSize'].astype('>f8'))[ix1]
max_rss_per_node_single = max_rss_per_node_single[numpy.invert(numpy.isnan(max_rss_per_node_single).astype(bool))]

# this is not right(yet), and might be hard to tease out.
#max_rss_per_cpu_mpi = (SACCT_obj['MaxRSS']*SACCT_obj['NNodes']/(SACCT_obj['NCPUS']))[SACCT_obj['NNodes']>1]
max_rss_per_cpu_mpi = (1./GB)*(SACCT_obj[MemVar].astype('>f8')*SACCT_obj['NTasks'].astype('>f8')/SACCT_obj['NCPUS'].astype('>f8'))[ix_mpi]
#max_rss_per_cpu_mpi = (1./GB)*(SACCT_obj['MaxVMSize'].astype('>f8')*SACCT_obj['NTasks'].astype('>f8')/SACCT_obj['NCPUS'].astype('>f8'))[ix_mpi]
max_rss_per_cpu_mpi = max_rss_per_cpu_mpi[numpy.invert(numpy.isnan(max_rss_per_cpu_mpi).astype(bool))]
#
# NOTE: "per_node" is really "per_task", for MPI or SPP/OMP jobs.
max_rss_per_node_mpi = (1./GB)*(SACCT_obj[MemVar].astype('>f8'))[ix_mpi]
#max_rss_per_node_mpi = (1./GB)*(SACCT_obj['MaxVMSize'].astype('>f8'))[ix_mpi]
max_rss_per_node_mpi = max_rss_per_node_mpi[numpy.invert(numpy.isnan(max_rss_per_node_mpi).astype(bool))]
#
max_rss_per_cpu_single = max_rss_per_cpu_single[ (max_rss_per_cpu_single < 16.)]
max_rss_per_cpu_mpi    = max_rss_per_cpu_mpi[ (max_rss_per_cpu_mpi < 16.)]
#
# quantiles:
qs = [.5, .75, .95]
qs_rss = [.5, .75, .90, .99]
#
qs_spp  = numpy.quantile(max_rss_per_cpu_single, qs_rss)
qs_mpi  = numpy.quantile(max_rss_per_cpu_mpi, qs_rss)
qs_node_spp = numpy.quantile(max_rss_per_node_single, qs_rss)
qs_node_mpi = numpy.quantile(max_rss_per_node_mpi, qs_rss)
#
fg_spp = plt.figure(figsize=(10,12))
fg_mpi = plt.figure(figsize=(10,12))
#
ax1 = fg_spp.add_subplot(2,2,1)
ax2 = fg_spp.add_subplot(2,2,2)
ax3 = fg_spp.add_subplot(2,2,3)
ax4 = fg_spp.add_subplot(2,2,4)
#
ax5 = fg_mpi.add_subplot(2,2,1)
ax6 = fg_mpi.add_subplot(2,2,2)
ax7 = fg_mpi.add_subplot(2,2,3)
ax8 = fg_mpi.add_subplot(2,2,4)
#
ax1.set_title(f'Single Task, {MemVar}/cpu PDF', size=16)
ax2.set_title(f'Single Task, {MemVar}/cpu CDF', size=16)
ax3.set_title(f'Single Task, {MemVar}/job PDF', size=16)
ax4.set_title(f'Single Task, {MemVar}/job CDF', size=16)
#
ax5.set_title(f'Multi-Task, {MemVar}/cpu PDF', size=16)
ax6.set_title(f"Multi-Task, {MemVar}/cpu CDF", size=16)
ax7.set_title(f'Multi-Task, {MemVar}/task PDF', size=16)
ax8.set_title(f'Multi-Task, {MemVar}/task CDF', size=16)
#
n_bins=50
#
h_single_cpu_pdf = ax1.hist(max_rss_per_cpu_single, bins=n_bins, density=True)
h_single_cpu_pdf = ax2.hist(max_rss_per_cpu_single, bins=n_bins, cumulative=True, density=True)
h_single_node_pdf = ax3.hist(max_rss_per_node_single, bins=n_bins, density=True)
h_single_node_cdf = ax4.hist(max_rss_per_node_single, bins=n_bins, cumulative=True, density=True)
#
h_mpi_cpu_pdf = ax5.hist(max_rss_per_cpu_mpi, bins=n_bins, density=True)
h_mpi_cpu_cdf = ax6.hist(max_rss_per_cpu_mpi, bins=n_bins*5, cumulative=True, density=True)
#
h_mpi_node_pdf = ax7.hist(max_rss_per_node_mpi, bins=n_bins, density=True)
h_mpi_node_cdf = ax8.hist(max_rss_per_node_mpi, bins=n_bins*5, cumulative=True, density=True)
#

for x,y in zip(qs_spp, qs_rss):
            ax2.plot([0., x, x], [y, y, 0.], ls='--', lw=2., label=f'{y*100.}th %: {x:.2f} GB/cpu' )
for x,y in zip(qs_node_spp, qs_rss):
    ax4.plot([0., x, x], [y, y, 0.], ls='--', lw=2., label=f'{y*100.}th %: {x:.2f} GB/job' )

for x,y in zip(qs_mpi, qs_rss):
    ax6.plot([0., x, x], [y, y, 0.], ls='--', lw=2., label=f'{y*100.}th %: {x:.2f} GB/cpu' )
    
for x,y in zip(qs_node_mpi, qs_rss):
    ax8.plot([0., x, x], [y, y, 0.], ls='--', lw=2., label=f'{y*100.}th %: {x:.2f} GB/task' )
        
for ax in (ax1, ax2, ax3, ax4, ax5, ax6,ax7,ax8):
    ax.grid()
    ax.legend(loc='lower right')
#
ax5.set_xlim(-.1,4.)
ax7.set_xlim(-.1,16.)

ax6.set_xlim(-.1,6.)
ax8.set_xlim(-.1, 15)
#

In [ ]:
print('** ', SACCT_obj.jobs_summary.dtype.names)

In [ ]:
j=0
for k,rw in enumerate(SACCT_obj):
    if rw['NTasks'] > 1:
        print(f'** ', rw[ ['JobID', 'NNodes', 'NCPUS', 'NTasks', 'ReqMem', 'MaxRSS', 'AveRSS'] ],
              " ** ", rw['MaxRSS']*rw['NTasks']/(GB*rw['NCPUS']),
              rw['MaxVMSize']*rw['NTasks']/(GB*rw['NCPUS']) )
        j+=1
    if j>100:
        break
    

## After-hours report(s)
Note the discriminators for weekend might not be quite right. Specifically, the tails of the weekend are not (I don't think...) being calculated correctly.

The approach of computing the time series and then using the modulus on the time series is a bit of a hack. It would probably be better to exactly break down each interval into time classes, then sum up.

But for now, this is probably a reasonabl estimate.

In [ ]:
hourlies = hpc_lib.get_cpu_hours(bin_size=7., d_t=1/24., verbose=0, 
                                 jobs_summary=SACCT_obj.jobs_summary)

morning_pct = .3
evening_pct = .7

ix_wknd   = numpy.logical_and( (hourlies['time'])%7 >= 2., (hourlies['time'])%7 <= 3. )
ix_afters = numpy.logical_or(hourlies['time']%1.<morning_pct , hourlies['time']%1>evening_pct)
# 
# "9 to 5!"
ix_dollies = numpy.logical_and(numpy.invert(ix_wknd), numpy.invert(ix_afters))

cpuh_totals = numpy.array([numpy.sum(hourlies['cpu_hours'][ix]) 
                           for ix in [ix_wknd, ix_afters, ix_dollies]])
cpuh_lbls = ['weekend', 'afters', 'dollies']
#
# we have calculated this with Dollies being .3 < t < .8,
#. so business hours and after-hours have equal weight. Note that this is not actually
#. 9-5, but seasonality reports suggested this was a reasonable definition, especially
#. with people in different time zones, and considering that generally research is not
#. contucted 9-5.
#
w_dollies = (evening_pct - morning_pct)*5.0
w_afters  = 5.0 - w_dollies
w_wknds   = 7.0 - (w_dollies + w_afters)

print(f'** weights:\n w_dollies: {w_dollies}\n  w_afters: {w_afters}\n  w_wknds: {w_wknds}')


cpuh_normalized = cpuh_totals/numpy.array([w_wknds, w_afters, w_dollies])/7.
#
print('** ** ', cpuh_totals)

#
fg = plt.figure(figsize=(12,8))
ax1 = fg.add_subplot(1,2,1)
ax2 = fg.add_subplot(1,2,2)
#
ax1.pie(cpuh_totals, labels=cpuh_lbls, autopct='%1.1f%%')
ax2.pie(cpuh_normalized, labels=cpuh_lbls, autopct='%1.1f%%')
#
ax1.set_title('9to5, raw')
ax2.set_title('9to5, normalized')